In [1]:
import pandas as pd
import numpy as np
import spacy
import json
from tqdm.notebook import tqdm
import glob
import plotly_express as px

from nlp_modules.ner.spacy_find_ners import find_ner_firms
from nlp_modules.ner.ner_firms_matching import get_most_possible_firm

In [ ]:
### Configs

# ticker symbols and their name variations
df_ner_firms = pd.read_excel("data/firms_NER_dataset_flatten.xlsx")
nlp_rus = spacy.load('ru_core_news_lg')
filter_value = 0.8

path_df = ".../1 Parsing/data"
# paths to all excels in the folder
all_paths = glob.glob(path_df + "/*.xlsx")
# concatenate all excels with parsed data
from_folder = []
for filepath in tqdm(all_paths):
    df = pd.read_excel(filepath, index_col=None, header=0)
    from_folder.append(df)

df = pd.concat(from_folder, axis=0, ignore_index=True)

  0%|          | 0/92 [00:00<?, ?it/s]

In [3]:
# Message count by day
df.groupby(['date'])['date'].count().reset_index(name='count')

,date,count
0,2024-05-01,34
1,2024-05-02,135
2,2024-05-03,112
3,2024-05-04,15
4,2024-05-05,28
...,...,...
87,2024-07-27,27
88,2024-07-28,22
89,2024-07-29,157
90,2024-07-30,195


In [4]:
df_process = df.copy().loc[:,['chat', 'date', 'time','text', 'text_id']]
df_process['chat'] = df['chat'].apply(lambda x: json.loads(x)['username'])

df_process['security_id'] = np.full(df_process.shape[0], None)
df_process['matching_score'] = np.full(df_process.shape[0], None)

df_process

,chat,date,time,text,text_id,security_id,matching_score
0,cozytrade_1,2024-05-01,21:22:22,☀️ Начинается набор в закрытый канал «Уютный к...,-1001485074571_6689,None,None
1,cozytrade_1,2024-05-01,14:56:24,🧩 Праздники - время фантазий.\n\n— 🔮 Вчера инд...,-1001485074571_6688,None,None
2,kuzmlab,2024-05-01,23:38:45,Любопытно прочитать свои посты двухлетней давн...,-1001241408755_7540,None,None
3,kuzmlab,2024-05-01,10:53:57,"Мир Труд Май всем, хотя с миром сейчас явная п...",-1001241408755_7539,None,None
4,mozgovikresearch,2024-05-01,22:00:47,Инвестиционно-спекулятивный портфель 30.04.202...,-1001298248713_1783,None,None
...,...,...,...,...,...,...,...
10805,razb0rka,2024-07-31,10:39:47,RAZB0RKA отчёта НОРНИКЕЛЬ по РСБУ 2кв'24. Банк...,-1001707166728_2645,None,None
10806,razb0rka,2024-07-31,08:36:46,МЕЧЕЛ результаты дочек по РСБУ 1п'24 🧮\n\nОтчё...,-1001707166728_2644,None,None
10807,truecon,2024-07-31,15:06:16,#еврозона #ЕЦБ #инфляция #ставки\n\nЕврозона: ...,-1001344892461_3759,None,None
10808,truecon,2024-07-31,08:38:40,#BOJ #Япония #ставки #JPY #инфляция \n\nБанк Я...,-1001344892461_3756,None,None


In [5]:
# Define NERs
for i in tqdm(range(df_process.shape[0])):

    if df_process.loc[i,'text'] is not np.nan:
        # found tickers
        ner_tickers = find_ner_firms(df_process.loc[i,'text'], nlp_rus)
        if ner_tickers:
            securities = []
            # each ticker should be counted only once, even if it occurs multiple times
            for ner in list(set(ner_tickers)):
                # obtain most matching tickers
                securities.append(get_most_possible_firm(ner, df_ner_firms, cols_to_match = ['securityid', 'firm_name']))  
            
            df_process.at[i,'security_id'] = [obj['securityid'] for obj in securities]
            df_process.at[i,'matching_score'] = [obj['matching_score'] for obj in securities]
        else:
            pass

  0%|          | 0/10810 [00:00<?, ?it/s]

In [7]:
df_process.head(5)

,chat,date,time,text,text_id,security_id,matching_score
0,cozytrade_1,2024-05-01,21:22:22,☀️ Начинается набор в закрытый канал «Уютный к...,-1001485074571_6689,[UKUZ],[0.6956521739130435]
1,cozytrade_1,2024-05-01,14:56:24,🧩 Праздники - время фантазий.\n\n— 🔮 Вчера инд...,-1001485074571_6688,"[SOFL, MOEX]","[0.5, 0.8]"
2,kuzmlab,2024-05-01,23:38:45,Любопытно прочитать свои посты двухлетней давн...,-1001241408755_7540,None,None
3,kuzmlab,2024-05-01,10:53:57,"Мир Труд Май всем, хотя с миром сейчас явная п...",-1001241408755_7539,[TCSG],[0.6111111111111112]
4,mozgovikresearch,2024-05-01,22:00:47,Инвестиционно-спекулятивный портфель 30.04.202...,-1001298248713_1783,None,None


In [8]:
# cleanly expand the cells containing multiple tickers into individual rows

# set the columns we want to keep unchanged as the index
df_process_indecies = df_process.set_index(['chat', 'date', 'time', 'text', 'text_id'])
df_process = df_process_indecies.apply(lambda x: x.explode()).reset_index().dropna()

filter_ = df_process['matching_score'] > 0.8 # filter_value
df_final = df_process[filter_]

In [9]:
df_final.head(5)

,chat,date,time,text,text_id,security_id,matching_score
16,birzhevikstocksofficial2,2024-05-01,20:57:05,"⚡️🇷🇺 Новость: под санкции Сша попала группа ""А...",-1001313313883_21401,ASTR,1.0
22,rynok_znania,2024-05-01,16:03:18,"Если серьезно, то, конечно же, в День труда не...",-1001823599276_4304,MOEX,0.875
41,invest_budka,2024-05-01,00:47:20,Итоги апреля 2024\n\nАпрель я провёл на работе...,-1001190535599_701,GAZP,0.933333
42,invest_budka,2024-05-01,00:47:20,Итоги апреля 2024\n\nАпрель я провёл на работе...,-1001190535599_701,SBER,1.0
45,razb0rka,2024-05-01,15:03:21,RAZB0RKA отчёта X5 по МСФО 1кв'24. Ускоряются!...,-1001707166728_2498,FIVE,1.0


In [ ]:
# save data
#df_final.to_excel('data/parsed_and_extracted_tickers.xlsx')

In [10]:
# ticker mention frequency chart: 
# shows the total number of mentions for each ticker to identify the most discussed securities.
df_1 = df_final.groupby(['security_id'])['security_id'].count().reset_index(name='count')

filter_= df_1['count'] > 10
df_1 = df_1[filter_]

fig = px.pie(df_1, values ='count', names = 'security_id', width=1000, height=700)

fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.show()

In [11]:
# ticker mention frequency chart: 
# displays the frequency of ticker mentions broken down by date and chat to analyze activity over time and across sources.
df_2 = df_final.groupby(['chat', 'security_id', 'date'])['security_id'].count().reset_index(name='count').sort_values(by='count', ascending=False)
df_2['chat_ticker_date_count'] = df_2['security_id'] + np.full(df_2.shape[0], '_') + \
    df_2['date'].astype(str) + np.full(df_2.shape[0], '_') + \
    df_2['chat'].astype(str) + np.full(df_2.shape[0], '_') + \
    df_2['count'].astype(str)
      

filter_ = df_2['count']>5
df_2 = df_2[filter_]
fig = px.pie(df_2, values ='count', width=1000, height=700, names = 'chat_ticker_date_count')

fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.show()